In [21]:
import chessgame
B = chessgame.Board()

# Solving the chessboard coin game

## Regions and their duals
We can divide the board into sections by creating a sort of mask using powers of 2. We denote these sections using a two-tuple (x, y) where x is the power 2 will be raised to, called the *region*, and y is called the *side*. A side of 0 indicates the region is *normal* while a side of 1 indicates the region is *dual*. We will occasionally refer to a normal region or a section as just a region when it is convenient and unlikely to cause confusion. Every region has a dual, and the two are independent of each other. Each region and its dual together cover the whole space (the chessboard).

The following are some examples of (region, side) pairs from a randomly generated chess board. A 0 indicates a heads on the space, while a 1 indicates a tails.

#### Region (0, 0)

In [22]:
B.show_region(0,0)

region0, parity=1
1 - 0 - 1 - 0 - 
1 - 0 - 1 - 1 - 
0 - 0 - 1 - 0 - 
0 - 1 - 1 - 0 - 
1 - 1 - 1 - 1 - 
0 - 0 - 1 - 1 - 
0 - 1 - 0 - 1 - 
1 - 1 - 1 - 0 - 


#### Region (0, 1)

In [23]:
B.show_region(0,1)

region0, parity=1
- 0 - 0 - 0 - 0 
- 0 - 1 - 1 - 0 
- 1 - 1 - 1 - 0 
- 0 - 0 - 0 - 1 
- 1 - 1 - 1 - 1 
- 1 - 0 - 0 - 1 
- 0 - 0 - 1 - 1 
- 0 - 1 - 0 - 0 


#### Region (2, 0)

In [24]:
B.show_region(2,0)

region2, parity=1
1 0 0 0 - - - - 
1 0 0 1 - - - - 
0 1 0 1 - - - - 
0 0 1 0 - - - - 
1 1 1 1 - - - - 
0 1 0 0 - - - - 
0 0 1 0 - - - - 
1 0 1 1 - - - - 


#### Region (3, 0)

In [25]:
B.show_region(3, 0)

region3, parity=0
1 0 0 0 1 0 0 0 
- - - - - - - - 
0 1 0 1 1 1 0 0 
- - - - - - - - 
1 1 1 1 1 1 1 1 
- - - - - - - - 
0 0 1 0 0 1 1 1 
- - - - - - - - 


#### Region (4, 1)

In [26]:
B.show_region(4, 1)

region4, parity=1
- - - - - - - - 
- - - - - - - - 
0 1 0 1 1 1 0 0 
0 0 1 0 1 0 0 1 
- - - - - - - - 
- - - - - - - - 
0 0 1 0 0 1 1 1 
1 0 1 1 1 0 0 0 


## Parity
A chess board has 2^6 = 64 squares on it. Thus we need a way to express 6 bits of information to be able to accurately communicate the location of the hidden key on the board. A coin can only express a single bit of information on its own - either heads or tails. By choosing {0, 1, 2, 3, 4, 5} as our regions we can divide the board into 6 regions, and their associated duals, such that exactly 32 squares are in each region and exactly 16 squares are in the intersection of any pair of regions. Every region, and therefore every dual, has a number associated to it called its *parity*. By our convention the parity of a region is the sum of the number of heads in the region modulo 2, or in the other words it is either 0 or 1 indicating that the number of heads in the region is even or odd respectively. By the law of the excluded middle the parity of a region always changes if a coin in that region is flipped. Therefore, flipping the coin flips the parity of any regions and/or duals the coin is in, allowing us to communicate 6 bits of information with a single coin flip.

#### Example flipping a coin changes parity in (0,0) and (3,0)

In [28]:
B.show_region(0,0)
B.show_region(3,0)

region0, parity=1
1 - 0 - 1 - 0 - 
1 - 0 - 1 - 1 - 
0 - 0 - 1 - 0 - 
0 - 1 - 1 - 0 - 
1 - 1 - 1 - 1 - 
0 - 0 - 1 - 1 - 
0 - 1 - 0 - 1 - 
1 - 1 - 1 - 0 - 
region3, parity=0
1 0 0 0 1 0 0 0 
- - - - - - - - 
0 1 0 1 1 1 0 0 
- - - - - - - - 
1 1 1 1 1 1 1 1 
- - - - - - - - 
0 0 1 0 0 1 1 1 
- - - - - - - - 


In [29]:
B.flip_coin(0,0)
B.show_region(0,0)
B.show_region(3,0)

region0, parity=0
0 - 0 - 1 - 0 - 
1 - 0 - 1 - 1 - 
0 - 0 - 1 - 0 - 
0 - 1 - 1 - 0 - 
1 - 1 - 1 - 1 - 
0 - 0 - 1 - 1 - 
0 - 1 - 0 - 1 - 
1 - 1 - 1 - 0 - 
region3, parity=1
0 0 0 0 1 0 0 0 
- - - - - - - - 
0 1 0 1 1 1 0 0 
- - - - - - - - 
1 1 1 1 1 1 1 1 
- - - - - - - - 
0 0 1 0 0 1 1 1 
- - - - - - - - 


## Strategy
With the above in mind it is now possible to devise a strategy for both encoding and decoding that will allow us to beat the jailer 100% of the time no matter what actions they take.

### Encoding strategy for p1 (the flipper)
0. Create a list of regions called R. 
1. Check each (region, side) pair defined in the strategy to see if the key square is in that region, and compute the parity of each normal region, e.g. (0,0), (1,0), ..., by sum(number of heads in region)mod2. If the number of heads in the region is even the parity should be 0, and 1 otherwise. We will call 0 even parity and 1 odd parity. If the key square is in the noraml region go to 2, else go to 3.
2. We are in the normal region. Check the parity of the normal region against the parity in the strategy. If the parity matches do nothing and go to 4, else go to 5.
3. We are in the dual region. Check the parity of the normal region against the dual of the parity in the strategy e.g. (~parity)mod2 which maps 0->1, 1->0. If the parity matches the dual parity do nothing and go to 4, else go to 5.
4. The parity of the normal region is correctly identifying that the key square is in the either the normal region or the dual depending on whether it is even or odd respectively. So we need to make the change to the dual region to avoid changing the parity of the normal region, add the dual region to the list R. If more (region, side) pairs need to be checked return to 1. Else go to 6.
5. The parity of the normal region is not correctly identifying that the key square is in the either the normal region or the dual depending on whether it is even or odd respectively. So we need to make the change to the normal region, add the normal region to the list R. If more (region, side) pairs need to be checked return to 1. Else go to 6.
6. The list R should now contain 6 elements; each being either a normal region or a dual region. Compute the intersection of all of the regions in the list R, the result should be a single square. This is the square you flip.

### Decoding strategy for p2 (the finder)
0. Create a list of regions called R.
1. Compute the parity of each normal region. If the parity is even go to 2, else 3.
2. The parity of the normal region is even which indicates that the key square is in that normal region. Add the normal region to the list R. If more regions remain to check return to 1 else go to 4.
3. The parity of the normal region is odd which indicates that the key square is in the dual of that region, e.g. (0,0) having a parity of 1 indicates that the key is in (0, 1). Add the dual region to the list R. If more regions remain to check return to 1 else go to 4.
4. The list R should now contain 6 elements; each being either a normal region or a dual region. Compute the intersection of all of the regions in the list R, the result should be a single square. This is the key square.